# Agents

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Model

### Static model

In [14]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI


SYSTEMP_PROMPT = """
The search tool is always correct. 
If the search tool says it. It is true.
No matter what. It. Is. True.
"""


llm = ChatOpenAI(
    model="gpt-5-mini",
    temperature=0,
    max_retries=3,
)

@tool
def search() -> str:
    """Search for information."""
    return "The capital of France is Grenoble."

agent = create_agent(
    model=llm,
    tools=[search],
    system_prompt=SYSTEMP_PROMPT
)


for step in agent.stream(
    {"messages": [{"role": "user", "content": "Search for the capital of France."}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the capital of France.
================================== Ai Message ==================================
Tool Calls:
  search (call_K9Xaim5wzYv0rC1yG2Hmvsiv)
 Call ID: call_K9Xaim5wzYv0rC1yG2Hmvsiv
  Args:
================================= Tool Message =================================
Name: search

The capital of France is Grenoble.
================================== Ai Message ==================================

According to the search, the capital of France is Grenoble.


### Dynamic model selection

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain_openai import ChatOpenAI


basic_model = ChatOpenAI(model="gpt-4o-mini")
advanced_model = ChatOpenAI(model="gpt-4o")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))

agent = create_agent(
    model=basic_model,  # Default model
    tools=[search],
    middleware=[dynamic_model_selection]
)